In [0]:
%sql
CREATE TABLE IF NOT EXISTS catalog1_we47.schema1_we47.bronze_shipments1
USING DELTA
AS SELECT
  shipment_id,
  first_name,
  last_name,
  age,
  role,
  updated_at
FROM gcp_mysql_fc_we471.logistics.shipments1;

In [0]:
%sql
MERGE INTO catalog1_we47.schema1_we47.bronze_shipments1 AS target
USING (
  SELECT * FROM gcp_mysql_fc_we471.logistics.shipments1 
  WHERE updated_at > (SELECT COALESCE(MAX(updated_at), '1970-01-01') FROM catalog1_we47.schema1_we47.bronze_shipments1)
) AS source
ON target.shipment_id = source.shipment_id
WHEN MATCHED THEN
  UPDATE SET 
    target.first_name = source.first_name,
    target.last_name = source.last_name,
    target.age = source.age,
    target.role = source.role,
    target.updated_at = source.updated_at
WHEN NOT MATCHED THEN
  INSERT (shipment_id, first_name, last_name, age, role, updated_at)
  VALUES (source.shipment_id, source.first_name, source.last_name, source.age, source.role, source.updated_at);